In [1]:
from transformers import BertModel,BertTokenizer

In [2]:
# model_path = "bert-base-uncased"

In [3]:
# model = BertModel.from_pretrained(model_path)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# tokenizer = BertTokenizer.from_pretrained(model_path)

In [5]:
# model.save_pretrained("../bert_model/")

In [7]:
# tokenizer.save_pretrained("../bert_model/")

#### Creating BERT Neural Network

In [8]:
import torch 
import torch.nn as nn

In [ ]:
class ToxicityModel(nn.Module):
    def __init__(self, bert_model_path):
        super(ToxicityModel,self).__init__()
        
        self.bert_model = BertModel.from_pretrained(bert_model_path)
        self.l1 = nn.Linear(768,256)  ## Reducing the Vector Dimension
        self.dropout = nn.Dropout(0.2)
        
        ## ['target','severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']
        
        
        
        
        